<h1>Project 4: SafeComm Digital Security Solutions</h1>
<p>Team Captain: E00020</p>
<p>Member 2: E00491</p>
<p>Member 3: E00045</p>
</br></br>
<h2>Table of Contents</h2>
<ol>
  <li><a href="#section1">Setup</a></li>
  <li><a href="#section2">EDA</a></li>
  <li><a href="#section3">Data Preprocessing</a></li>
  <li><a href="#section4">Models</a></li>
  <li><a href="#section5">Testing</a></li>
</ol>
</br></br>

<h3 id="section1">Setup</h3>

In [ ]:
# importing all necessarylibaries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import spacy
import re
import string
import unicodedata
from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from textblob import TextBlob
from textblob import Word
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings('ignore')

# importing the csv into a pandas data frame
df = pd.read_csv("./sms.csv")


<h3 id="section2">Exploratory Data Analysis<h3>


In [ ]:
# visualisation


<h3 id="section3">Data Preprocessing</h3>

In [ ]:
# tokenising ect


In [ ]:
# handle missing values


In [ ]:
# training-test-validation split


<h3 id="section4">Models</h3>

In [ ]:
# model 1


In [ ]:
# model 2


In [ ]:
# model 3


<h3 id="section5">Testing</h3>

In [ ]:
# alt run tests in seperate file?
